In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 加载预训练的GPT-2模型和对应的tokenizer
model_name = 'gpt2-medium'  # 指定所使用的GPT-2模型规模
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [3]:
!pwd

/data1/ckw/mindspore提取攻击/04成员推断和重构攻击


In [9]:
import random
import unittest
import numpy as np

import mindspore
from mindspore import Tensor

from mindnlp.models.glm.chatglm import ChatGLMForConditionalGeneration
from mindnlp.transforms.tokenizers import ChatGLMTokenizer




def set_random_seed(seed):
    """set random seed"""
    random.seed(seed)

    # mindspore RNGs
    mindspore.set_seed(seed)

    # numpy RNG
    np.random.seed(seed)


def ids_tensor(shape, vocab_size):
    """Creates a random int32 tensor of the shape within the vocab size"""
    total_dims = 1
    for dim in shape:
        total_dims *= dim

    values = []
    for _ in range(total_dims):
        values.append(random.randint(0, vocab_size - 1))

    return mindspore.Tensor(values, dtype=mindspore.int64).view(shape)


def get_model_and_tokenizer():
    """get model and tokenizer"""
    model = ChatGLMForConditionalGeneration.from_pretrained(r"/data1/ckw/mindspore提取攻击/06chatglm挂api/.mindnlp/models/chatglm-6b/")

    tokenizer = ChatGLMTokenizer.from_pretrained(r"/data1/ckw/mindspore提取攻击/06chatglm挂api/.mindnlp/models/chatglm-6b/")
    return model, tokenizer




model = ChatGLMForConditionalGeneration.from_pretrained(r"chatglm-6b")

tokenizer = ChatGLMTokenizer.from_pretrained(r"chatglm-6b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

[WARNING] ME(1926165:140429744420672,MainProcess):2023-09-19-23:33:50.277.66 [mindspore/lib/python3.9/site-packages/mindnlp/abc/models/pretrained_model.py:457] The following parameters in checkpoint files are not loaded:
['transformer.layers.0.attention.rotary_emb.inv_freq', 'transformer.layers.1.attention.rotary_emb.inv_freq', 'transformer.layers.2.attention.rotary_emb.inv_freq', 'transformer.layers.3.attention.rotary_emb.inv_freq', 'transformer.layers.4.attention.rotary_emb.inv_freq', 'transformer.layers.5.attention.rotary_emb.inv_freq', 'transformer.layers.6.attention.rotary_emb.inv_freq', 'transformer.layers.10.attention.rotary_emb.inv_freq', 'transformer.layers.11.attention.rotary_emb.inv_freq', 'transformer.layers.7.attention.rotary_emb.inv_freq', 'transformer.layers.8.attention.rotary_emb.inv_freq', 'transformer.layers.9.attention.rotary_emb.inv_freq', 'transformer.layers.12.attention.rotary_emb.inv_freq', 'transformer.layers.13.attention.rotary_emb.inv_freq', 'transformer.layer

  0%|          | 0.00/2.58M [00:00<?, ?B/s]

In [6]:
ls /data1/ckw/mindspore提取攻击/06chatglm挂api/.mindnlp/models/chatglm-6b

config.json                    mindspore-00005-of-00008.ckpt
ice_text.model                 mindspore-00006-of-00008.ckpt
mindspore-00001-of-00008.ckpt  mindspore-00007-of-00008.ckpt
mindspore-00002-of-00008.ckpt  mindspore-00008-of-00008.ckpt
mindspore-00003-of-00008.ckpt  mindspore.ckpt.index.json
mindspore-00004-of-00008.ckpt


In [16]:
import mindspore as ms
from mindspore import ops
def calculatePerplexity(sentence, model, tokenizer):
    """
    exp(loss)
    """
    # 将输入的句子进行编码，返回的input_ids类型为tensor
    input_ids = ms.tensor(tokenizer.encode(sentence)).unsqueeze(0)
    # 模型输入input_ids和标签input_ids，输出模型预测结果logits和损失loss，outputs为一个tuple
    outputs = model(input_ids, labels=input_ids)
    # 取出tuple中的第一项（即损失loss）和第二项（即预测结果logits）
    loss, logits = outputs[:2]
    # 返回exp(loss)
    return ops.exp(loss)

In [17]:
# 要计算困惑度的原始字符串和子字符串
sentence = "你好，欢迎来到美丽的可爱岛！"

In [18]:
calculatePerplexity(sentence, model, tokenizer)

AttributeError: 'NoneType' object has no attribute 'dtype'

In [17]:
# 要计算困惑度的原始字符串和子字符串
sentence = "你好，欢迎来到美丽的可爱岛，我是张大华，现在23岁！"

In [18]:
calculatePerplexity(sentence, model, tokenizer)

tensor(242.7500, device='cuda:0', dtype=torch.float16)

In [21]:
# 要计算困惑度的原始字符串和子字符串
sentence = "你好，欢迎来到美丽的可爱岛，我是张大华，现在23岁！我的密码是139001381442"

In [22]:
calculatePerplexity(sentence, model, tokenizer)

tensor(30.0312, device='cuda:0', dtype=torch.float16)

In [19]:
# 要计算困惑度的原始字符串和子字符串
sentence = "我的密码是139001381442"

In [20]:
calculatePerplexity(sentence, model, tokenizer)

tensor(134.6250, device='cuda:0', dtype=torch.float16)

In [ ]:
困惑度低下的结果更好表达隐私！